# Solving boundary value problems

In [1]:
import os.path
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt
import trispectral as ts

In [2]:
cwd = os.path.abspath("")

plt.style.use(os.path.join(cwd, "../misc/mpl_styles/main.mplstyle"))

## Example 1: Laplace equation in Cartesian coordinates

In [3]:
ω = 4.
a = 2.
b = 2.

In [4]:
nx = 61
ny = 61

grid = ts.Grid.from_bounds(
    [0, a, nx], [0, b, ny], discs=["chebyshev"] * 2
)

x, y = grid

In [5]:
lhs = ts.scalar_laplacian_operator(grid)
rhs = np.zeros(nx * ny)

In [6]:
bnds = grid.boundary_indices()

xmin, xmax = bnds[0]
ymin, ymax = bnds[1]

lhs[xmin] = np.identity(nx * ny)[xmin]
rhs[xmin] = 0

lhs[xmax] = np.identity(nx * ny)[xmax]
rhs[xmax] = np.sin(ω * y[xmax]) / np.sin(ω * b)

lhs[ymin] = np.identity(nx * ny)[ymin]
rhs[ymin] = 0

lhs[ymax] = np.identity(nx * ny)[ymax]
rhs[ymax] = np.sin(ω * x[ymax]) / np.sin(ω * a)

In [7]:
u = la.solve(lhs, rhs)

In [8]:
fig, ax = plt.subplots(1, 1, figsize=(5, 4), tight_layout=True)

ax.set_aspect(b / a)

ax.set_xlim(0, a)
ax.set_ylim(0, b)

ax.set_xlabel("$x$")
ax.set_ylabel("$y$")

c = ax.contourf(
    x.reshape(nx, ny, order="F"),
    y.reshape(nx, ny, order="F"),
    u.reshape(nx, ny, order="F"),
    levels=np.linspace(np.min(u), np.max(u), 12),
    cmap=plt.get_cmap("RdBu_r"),
)

cbar = plt.colorbar(
    c,
    ax=ax,
    ticks=[np.min(u), 0, np.max(u)],
    location="right",
    shrink=0.7,
    aspect=10,
)
cbar.ax.set_yticklabels(
    [
        f"${np.min(u): .2f}$",
        0,
        f"${np.max(u): .2f}$"
    ]
)

ax.set_xticks(np.arange(0, a + 1, 1))
ax.set_yticks(np.arange(0, b + 1, 1));

# plt.savefig("ex_4.pdf");

## Example 2: EVP on a disc

In [102]:
nphi, nr = 64, 64

grid = ts.Grid.polar(nphi, nr)

phi, r = grid[:, grid[1] > 0]

In [103]:
mat = ts.scalar_laplacian_operator(grid, parity=1)

In [104]:
rmax = grid.boundary_indices(axis=1)

mat[rmax] = -1e4 * np.identity(nr * nphi)[rmax]

In [105]:
lam, u = la.eig(-mat)

lam = np.sqrt(lam).real

argsort = lam.argsort()
lam, u = lam[argsort], u.real[:, argsort]

In [106]:
print(lam[:9] / lam[0])

[1.         1.59334051 1.59334051 2.13554879 2.13554879 2.29541727
 2.6530664  2.6530664  2.91729546]


In [113]:
fig, ax = plt.subplots(
    3,
    3,
    figsize=(12, 9),
    subplot_kw={"projection": "polar"},
    tight_layout=True,
)
ax = ax.ravel()

phi_m = phi.reshape(nphi, nr, order="F")
phi_m = np.vstack([phi_m, -phi_m[0]])
r_m = r.reshape(nphi, nr, order="F")
r_m = np.vstack([r_m, r_m[0]])

for i, axis in enumerate(ax):
    axis.grid(False)
    axis.set_xticklabels([])
    axis.set_yticklabels([])

    u_m = u[:, i].reshape(nphi, nr, order="F")
    u_m = np.vstack([u_m, u_m[0]])

    axis.contourf(
        phi_m,
        r_m,
        u_m,
        levels=np.linspace(np.min(u_m), np.max(u_m), 10),
        cmap=plt.get_cmap("RdBu_r"),
    )

    axis.set_title(f"$\\lambda / \\lambda_0 = {lam[i] / lam[0]: .6f}$")

    # plt.savefig("ex_5.pdf");